# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests


# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,296.99,66,100,2.68,PN,1682625647
1,1,avarua,-21.2078,-159.7750,299.18,69,100,2.57,CK,1682625920
2,2,alice springs,-23.7000,133.8833,289.90,88,100,3.08,AU,1682625920
3,3,albany,42.6001,-73.9662,288.58,48,26,0.89,US,1682625763
4,4,aykhal,66.0000,111.5000,261.16,100,95,1.76,RU,1682625921


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Convert 'Max Temp' from Kelvin to Celsius
city_data_df['Max Temp'] = city_data_df['Max Temp'] - 273.15

# Narrow down cities that fit criteria and drop any results with null values
city_filtered = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & 
    (city_data_df["Max Temp"] > 21) & 
    (city_data_df["Wind Speed"] < 4.5) & 
    (city_data_df["Cloudiness"] == 0)
]


# Drop any rows with null values
city_filtered = city_filtered.dropna()


# Display sample data
city_filtered

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
188,188,ormara,25.2088,64.6357,26.13,77,0,3.60,PK,1682626006
224,224,washougal,45.5826,-122.3534,24.32,55,0,4.12,US,1682626013
278,278,san rafael,-34.6177,-68.3301,26.30,29,0,4.17,AR,1682626034
294,294,khark,29.2614,50.3306,24.83,72,0,4.08,IR,1682626045
352,352,francisco i. madero,29.7059,-107.8043,23.35,10,0,1.59,MX,1682626072
354,354,pathein,16.7833,94.7333,25.75,81,0,2.44,MM,1682626073


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_filtered[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.assign(**{'Hotel Name': ''})

# Display sample data
print(hotel_df)

                    City Country      Lat       Lng  Humidity Hotel Name
188               ormara      PK  25.2088   64.6357        77           
224            washougal      US  45.5826 -122.3534        55           
278           san rafael      AR -34.6177  -68.3301        29           
294                khark      IR  29.2614   50.3306        72           
352  francisco i. madero      MX  29.7059 -107.8043        10           
354              pathein      MM  16.7833   94.7333        81           


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
# Define the Geoapify Places API endpoint and parameters
base_url = "https://api.geoapify.com/v1/places"
api_key = geoapify_key
search_type = "hotels"
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']
    
    Lat = hotel_df["Lat"]
    Lng = hotel_df["Lng"]
    
    params = {
        "apiKey": api_key,
        "type": search_type,
        "lat": lat,
        "lng": lng,
        "radius": radius
    }
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"point:{Lat},{Lng}"
    params["bias"] = f"point:{Lng},{Lat}"
    
     # Send the API request and parse the response
    response = requests.get(base_url, params=params)
    results = response.json().get('features')

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    if results is not None and len(results) > 0:
        hotel_df.loc[index, "Hotel Name"] = results[0]["properties"]["name"]
    else: 
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ormara - nearest hotel: No hotel found
washougal - nearest hotel: No hotel found
san rafael - nearest hotel: No hotel found
khark - nearest hotel: No hotel found
francisco i. madero - nearest hotel: No hotel found
pathein - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
188,ormara,PK,25.2088,64.6357,77,No hotel found
224,washougal,US,45.5826,-122.3534,55,No hotel found
278,san rafael,AR,-34.6177,-68.3301,29,No hotel found
294,khark,IR,29.2614,50.3306,72,No hotel found
352,francisco i. madero,MX,29.7059,-107.8043,10,No hotel found
354,pathein,MM,16.7833,94.7333,81,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:

# Configure the map plot
hotels_map = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols=["Hotel Name", "Country"]
)


# Display the map
hotels_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)